In [1]:
pip install torch numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# I will create the model that predict the crop yields for apples and oranges by looking at the temp, rainfall, humidity in region. the targeted variable should be weighted sum and bias.

In [2]:
# yield_apple= w11+ tem +w12 + rainfall + w13 + humidity +b1
# yield_oranges =w21+ tem +w22 + rainfall + w23 + humidity +b2

In [4]:
import numpy as np 
import torch

# Training datasets

In [9]:
# input tem, rainfall, humidity
inputs= np.array([[73, 67, 43],
                  [91, 88, 64],
                  [87, 134, 58],
                  [102, 43, 37],
                  [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [10]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [21]:
# Weights and biases
torch.manual_seed(100)
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.3607, -0.2859, -0.3938],
        [ 0.2429, -1.3833, -2.3134]], requires_grad=True)
tensor([-0.3172, -0.8660], requires_grad=True)


In [22]:
# for transpose matrix we use .t() methode @ means multiply
inputs @  w.t() +b


tensor([[ -10.0802, -175.2920],
        [ -17.8629, -248.5511],
        [ -30.0950, -299.2751],
        [   9.6038, -121.1664],
        [ -30.4476, -278.8429]], grad_fn=<AddBackward0>)

In [23]:
# for transpose matrix we use .t() methode @ means multiply
def model(x):
    return x @ w.t() + b

In [24]:
# Generate predictions
predict = model(inputs)
print(predict)

tensor([[ -10.0802, -175.2920],
        [ -17.8629, -248.5511],
        [ -30.0950, -299.2751],
        [   9.6038, -121.1664],
        [ -30.4476, -278.8429]], grad_fn=<AddBackward0>)


In [25]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# make a loss function 
to find the best possible way for this we can diffrenece them of targeted and predicted tensors also we can calculate the whole part as mean square error for this we can use the numel() function to count the total elements in tensors.

In [26]:
# MSE loss  use the t1 as predicted and t2 as targeted tensors
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

# Now compute the loss 
loss=mse(predict,targets)
print(loss)

tensor(60684.3203, grad_fn=<DivBackward0>)


# We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount.

In [33]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [34]:

print(w.grad)
print(b.grad)

tensor([[ -7452.7891,  -9290.0107,  -5523.2363],
        [-26181.5059, -30022.7988, -18338.3887]])
tensor([ -91.9764, -316.6255])


In [41]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [36]:
# Let's verify that the loss is actually lower
loss = mse(predict, targets)
print(loss)

tensor(60684.3203, grad_fn=<DivBackward0>)


In [38]:
'''Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates 
gradients. Otherwise, the next time we invoke .backward on the loss, 
the new gradient values are added to the existing gradients, which may lead to unexpected results.'''
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [39]:
predict=model(inputs)
print(predict)

tensor([[  18.0013,  -81.0591],
        [  19.1231, -124.5811],
        [  14.1789, -151.9793],
        [  36.8860,  -28.3598],
        [   5.4084, -159.3885]], grad_fn=<AddBackward0>)


In [43]:
loss=mse(predict,targets)
loss

tensor(26269.6777, grad_fn=<DivBackward0>)

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch

In [74]:
# train for 100 epoche << better than train on 400 epoche
for i in range(400):
    predict=model(inputs)
    loss=mse(predict,targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()


In [75]:
# Calculate loss verify the loss now lower ?
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(4.3866, grad_fn=<DivBackward0>)


In [76]:
predict

tensor([[ 57.1850,  70.3897],
        [ 81.6272,  98.8857],
        [119.9785, 136.8845],
        [ 21.4459,  38.1266],
        [100.6367, 115.2171]], grad_fn=<AddBackward0>)

In [77]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])